In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,6798
2,Huelva,Confirmados PDIA 14 días,2559
3,Huelva,Tasa PDIA 14 días,"501,0347669963171"
4,Huelva,Confirmados PDIA 7 días,1263
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,16
629,Municipio de Huelva sin especificar,Total Confirmados,110
630,Municipio de Huelva sin especificar,Curados,21


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  6798.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  1863.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 382 personas en los últimos 7 días 

Un positivo PCR cada 195 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,6798.0,1263.0,2559.0,510743.0,247.286796,501.034767,367.0
Huelva-Costa,3537.0,739.0,1391.0,288115.0,256.494802,482.793329,209.0
Condado-Campiña,2130.0,340.0,748.0,155057.0,219.274202,482.403245,110.0
Huelva (capital),1863.0,376.0,738.0,143663.0,261.723617,513.702206,105.0
Lepe,551.0,129.0,229.0,27431.0,470.270861,834.821917,47.0
Sierra de Huelva-Andévalo Central,1022.0,168.0,378.0,67571.0,248.627370,559.411582,45.0
Moguer,160.0,52.0,83.0,22088.0,235.421949,375.769649,32.0
Palos de la Frontera,106.0,33.0,55.0,11289.0,292.319957,487.199929,19.0
Valverde del Camino,219.0,46.0,104.0,12820.0,358.814353,811.232449,18.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Aracena,255.0,68.0,151.0,8107.0,838.781300,1862.587887,9.0
Escacena del Campo,85.0,7.0,30.0,2284.0,306.479860,1313.485114,0.0
Gibraleón,289.0,43.0,141.0,12607.0,341.080352,1118.426271,4.0
Palma del Condado (La),311.0,49.0,115.0,10761.0,455.348016,1068.673915,15.0
Villablanca,37.0,22.0,27.0,2848.0,772.471910,948.033708,3.0
Lepe,551.0,129.0,229.0,27431.0,470.270861,834.821917,47.0
Valverde del Camino,219.0,46.0,104.0,12820.0,358.814353,811.232449,18.0
Niebla,70.0,12.0,30.0,4117.0,291.474375,728.685936,2.0
San Silvestre de Guzmán,5.0,1.0,4.0,623.0,160.513644,642.054575,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Alosno,35.0,1.0,15.0,3923.0,25.490696,382.360438,0.0,0.066667
Calañas,34.0,1.0,9.0,2791.0,35.829452,322.465066,0.0,0.111111
Cortegana,123.0,1.0,7.0,4666.0,21.431633,150.021432,0.0,0.142857
Cumbres Mayores,10.0,1.0,6.0,1755.0,56.980057,341.880342,0.0,0.166667
Escacena del Campo,85.0,7.0,30.0,2284.0,306.479860,1313.485114,0.0,0.233333
Higuera de la Sierra,12.0,1.0,4.0,1287.0,77.700078,310.800311,0.0,0.250000
San Silvestre de Guzmán,5.0,1.0,4.0,623.0,160.513644,642.054575,0.0,0.250000
Almonaster la Real,30.0,2.0,7.0,1819.0,109.950522,384.826828,0.0,0.285714
Gibraleón,289.0,43.0,141.0,12607.0,341.080352,1118.426271,4.0,0.304965
